In [23]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
cosine_similarity(np.array([4,0,5,3,5,0,0]).reshape(1,-1),\
                  np.array([0,4,0,4,0,5,0]).reshape(1,-1))

array([[0.18353259]])

In [25]:
cosine_similarity(np.array([4,0,5,3,5,0,0]).reshape(1,-1),\
                  np.array([2,0,2,0,1,0,0]).reshape(1,-1))

array([[0.88527041]])

In [26]:
cosine_similarity(np.array([-.25,0,.75,-1.25,.75,0,0])\
                  .reshape(1,-1),\
                  np.array([0,-.33,0,-.33,0,.66,0])\
                  .reshape(1,-1))

array([[0.30772873]])

In [27]:
cosine_similarity(np.array([-.25,0,.75,-1.25,.75,0,0])\
                  .reshape(1,-1),\
                  np.array([.33,0,.33,0,-.66,0,0])\
                  .reshape(1,-1))

array([[-0.24618298]])

In [28]:
user_x = [0,.33,0,-.66,0,33,0]
user_y = [0,0,0,-1,0,.5,.5]

cosine_similarity(np.array(user_x).reshape(1,-1),\
                  np.array(user_y).reshape(1,-1))

array([[0.42447212]])

In [29]:
user_x = [0,.33,0,-.66,0,33,0]
user_z = [0,-.125,0,-.625,0,.375,.375]

cosine_similarity(np.array(user_x).reshape(1,-1),\
                  np.array(user_z).reshape(1,-1))

array([[0.46571861]])

In [30]:
s1 = [-1.0,0.0,0.0,0.0,1.0]
s2 = [-1.66,0.0,.33,0.0,1.33]

cosine_similarity(np.array(s1).reshape(1,-1),\
                  np.array(s2).reshape(1,-1))

array([[0.98221439]])

## Build a Recommendation Engine

In [31]:
myun = 'whyang78'
mypw = 'a116f1f58e90d9b03147726e6afb8044d6b154b6'

### Get the repos I have starred

In [32]:
my_starred_repos = []
def get_starred_by_me():
    resp_list = []
    last_resp = ''
    first_url_to_get = 'https://api.github.com/user/starred'
    first_url_resp = requests.get(first_url_to_get, auth=(myun,mypw))
    last_resp = first_url_resp
    resp_list.append(json.loads(first_url_resp.text))
    
    while last_resp.links.get('next'):
        next_url_to_get = last_resp.links['next']['url']
        next_url_resp = requests.get(next_url_to_get, auth=(myun,mypw))
        last_resp = next_url_resp
        resp_list.append(json.loads(next_url_resp.text))
        
    for i in resp_list:
        for j in i:
            msr = j['html_url']
            my_starred_repos.append(msr)

In [33]:
get_starred_by_me()

ConnectionError: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /user/starred (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002848A62D860>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [ ]:
my_starred_repos

In [ ]:
len(my_starred_repos)

In [ ]:
my_starred_users = []
for ln in my_starred_repos:
    right_split = ln.split('.com/')[1]
    starred_usr = right_split.split('/')[0]
    my_starred_users.append(starred_usr)

In [ ]:
my_starred_users

In [ ]:
len(my_starred_users)

### Looks like some duplication because I starred multiple repos for some users

In [ ]:
len(set(my_starred_users))

### Now let's get the repos they starred

In [ ]:
starred_repos = {k:[] for k in set(my_starred_users)}
def get_starred_by_user(user_name):
    starred_resp_list = []
    last_resp = ''
    first_url_to_get = 'https://api.github.com/users/'+ user_name +'/starred'
    first_url_resp = requests.get(first_url_to_get, auth=(myun,mypw))
    last_resp = first_url_resp
    starred_resp_list.append(json.loads(first_url_resp.text))
    
    while last_resp.links.get('next'):
        next_url_to_get = last_resp.links['next']['url']
        next_url_resp = requests.get(next_url_to_get, auth=(myun,mypw))
        last_resp = next_url_resp
        starred_resp_list.append(json.loads(next_url_resp.text))
        
    for i in starred_resp_list:
        for j in i:
            sr = j['html_url']
            starred_repos.get(user_name).append(sr)

In [ ]:
for usr in list(set(my_starred_users)):
    print(usr)
    try:
        get_starred_by_user(usr)
    except:
        print('failed for user', usr)

In [ ]:
len(starred_repos)

### Now we'll build a vocabulary that includes all the repos starred by the users I starred

In [ ]:
repo_vocab = [item for sl in list(starred_repos.values()) for item in sl]

In [ ]:
repo_set = list(set(repo_vocab))

In [ ]:
len(repo_set)

In [ ]:
all_usr_vector = []
for k,v in starred_repos.items():
    usr_vector = []
    for url in repo_set:
        if url in v:
            usr_vector.extend([1])
        else:
            usr_vector.extend([0])
    all_usr_vector.append(usr_vector)

In [ ]:
len(all_usr_vector)

In [ ]:
df = pd.DataFrame(all_usr_vector, columns=repo_set, index=starred_repos.keys())

In [ ]:
df

In [ ]:
len(df.columns)

### Now I need to add myself to this dataframe to find the similarity between myself and the other users

In [ ]:
my_repo_comp = []
for i in df.columns:
    if i in my_starred_repos:
        my_repo_comp.append(1)
    else:
        my_repo_comp.append(0)

In [ ]:
mrc = pd.Series(my_repo_comp).to_frame(myun).T

In [ ]:
mrc

In [ ]:
mrc.columns = df.columns

In [ ]:
fdf = pd.concat([df, mrc])

In [ ]:
fdf

In [ ]:
l2 = my_starred_repos

In [ ]:
l1 = fdf.iloc[-1,:][fdf.iloc[-1,:]==1].index.values

In [ ]:
a = set(l1)
b = set(l2)

In [ ]:
b.difference(a)

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from scipy.stats import pearsonr

In [ ]:
sim_score = {}
for i in range(len(fdf)):
    ss = pearsonr(fdf.iloc[-1,:], fdf.iloc[i,:])
    sim_score.update({i: ss[0]})

In [ ]:
sf = pd.Series(sim_score).to_frame('similarity')

In [ ]:
sf

In [ ]:
sf.sort_values('similarity', ascending=False)

In [ ]:
fdf.index[0]

In [ ]:
all_recs = fdf.iloc[[0,26,19,25],:][fdf.iloc[[0,26,19,25],:]==1].fillna(0).T

In [ ]:
all_recs[(all_recs==1).all(axis=1)]

In [ ]:
str_recs_tmp = all_recs[all_recs[myun]==0].copy()
str_recs = str_recs_tmp.iloc[:,:-1].copy()
str_recs

In [ ]:
str_recs[(str_recs==1).all(axis=1)]

In [ ]:
str_recs[str_recs.sum(axis=1)>1]